In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

feedback_produto = """
Estou muito satisfeito com o Smartphone XYZ Pro. O desempenho é excelente, e o sistema 
operacional é rápido e intuitivo. A câmera é um dos principais destaques, especialmente o 
modo noturno, que captura imagens incríveis mesmo em baixa iluminação. A duração da bateria 
também impressiona, durando facilmente um dia inteiro com uso intenso.
Por outro lado, sinto que o produto poderia ser melhorado em alguns aspectos. A tela, 
embora tenha cores vibrantes, parece refletir bastante luz, dificultando o uso sob o sol. 
Além disso, o carregador incluído na caixa não oferece carregamento rápido, o que é um ponto 
negativo considerando o preço do aparelho
"""


review_template = ChatPromptTemplate.from_template("""
Para o texto a seguir, extraia as seguintes informações:
produto: Nome do produto mencionado no texto.
características_positivas: Liste todas as características positivas mencionadas sobre o produto.
características_negativas: Liste todas as características negativas mencionadas sobre o produto.
recomendação: O cliente recomenda o produto? Responda True para sim ou False para não.

Texto: {review}

Retorne a resposta no formato JSON
"""
)

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
# response = chat.invoke(review_template.format_prompt(review=feedback_produto))
# print(response.content)

schema_produto = ResponseSchema(
    name="produto",
    type="string",
    description="Nome do produto mencionado no texto."
)

schema_caracteristicas_positivas = ResponseSchema(
    name="características_positivas",
    type="list",
    description="Liste todas as características positivas mencionadas sobre o produto."
)

schema_caracteristicas_negativas = ResponseSchema(
    name="características_negativas",
    type="list",
    description="Liste todas as características negativas mencionadas sobre o produto."
)

schema_recomendacao = ResponseSchema(
    name="recomendação",
    type="bool",
    description="O cliente recomenda o produto? Responda True para sim ou False para não."
)

schemas = [
    schema_produto,
    schema_caracteristicas_positivas,
    schema_caracteristicas_negativas,
    schema_recomendacao
]

parser = StructuredOutputParser.from_response_schemas(schemas)
schema_formatted = parser.get_format_instructions()

# print(schema_formatted)

review_template2 = ChatPromptTemplate.from_template("""
Para o texto a seguir, extraia as seguintes informações:
produto, caracteristicas_positivas, caracteristicas_negativas e recomendacao

Texto: {review}

{schema}
""", partial_variables={"schema":schema_formatted}
)

response = chat.invoke(review_template2.format_prompt(review=feedback_produto))
print(response.content)

jsonResponse = parser.parse(response.content) # type: ignore
print(jsonResponse["produto"])

```json
{
	"produto": "Smartphone XYZ Pro",
	"características_positivas": ["Desempenho excelente", "Sistema operacional rápido e intuitivo", "Câmera de destaque, especialmente o modo noturno", "Duração da bateria impressionante"],
	"características_negativas": ["Tela reflete bastante luz", "Carregador não oferece carregamento rápido"],
	"recomendação": true
}
```
Smartphone XYZ Pro
